In [ ]:
import numpy as np
from scipy.special import expit
import matplotlib.pyplot as plt

np.random.seed(seed=0)

# Data

Linearly separable dataset consisting of datapoints generated from two Gaussian distributions.

mean_1 = [-5, 2]
cov_1 = [[1,0.5],[0.5,3]]
gaussian_1 = np.random.multivariate_normal(mean_1, cov_1, 50)

mean_2 = [5, -2]
cov_2 = [[1,0.5],[0.5,3]]
gaussian_2 = np.random.multivariate_normal(mean_2, cov_2, 50)

plt.scatter(gaussian_1[:,0], gaussian_1[:,1])
plt.scatter(gaussian_2[:,0], gaussian_2[:,1])
plt.xlim(-10,10)
plt.ylim(-10,10)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
mean = [0, 0]
cov = [[5,0],[0,5]]
gaussian = np.random.multivariate_normal(mean, cov, 50)

angle = 2.0*np.pi*np.random.random(50)
radius = 6 + np.random.choice([1,-1], 50)* 2.0*np.random.random(50)

xs = radius * np.cos(angle)
ys = radius * np.sin(angle)

plt.scatter(gaussian[:,0], gaussian[:,1])
plt.scatter(xs, ys)
plt.xlim(-10,10)
plt.ylim(-10,10)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

### Sort data into labeled dataset

The dataset is represented as an (N,2) array with each row $n$ corresponding to input variable vector $x^{(n)} = (x^{(n)}_1, x^{(n)}_2)$.

<table style="border:1">
    <tr>
        <td>$x^{(1)}_1$</td>
        <td>$x^{(1)}_2$</td>
    </tr>
    <tr>
        <td>$x^{(2)}_1$</td>
        <td>$x^{(2)}_2$</td>
    </tr>
        <tr>
        <td>$\dots$</td>
        <td>$\dots$</td>
    </tr>
    <tr>
        <td>$x^{(N)}_1$</td>
        <td>$x^{(N)}_2$</td>
    </tr>
</table>

data_1 = np.zeros((gaussian_1.shape[0], 3))
data_1[:,0:2] = gaussian_1

data_2 = np.ones((gaussian_2.shape[0], 3))
data_2[:,0:2] = gaussian_2

In [ ]:
data_1 = np.zeros((gaussian.shape[0], 3))
data_1[:,0:2] = gaussian

data_2 = np.ones((xs.shape[0], 3))
data_2[:,0] = xs
data_2[:,1] = ys

In [ ]:
# Dataset is stored as a (N,2) array
dataset = np.vstack((data_1, data_2))
dataset = np.random.permutation(dataset)

# Number of samples
N = dataset.shape[0]

### Linear basis function model

Applying a fixed nonlinear transformation on the input variable vectors $x$ to generate feature vectors $\phi$, allowing the model to consider linear combinations of fixed nonlinear functions of the input variables.

$\phi^{(n)} = (\phi_1(x^{(n)}), \dots, \phi_N(x^{(n)})), \:\:\: where \: \phi_j = f(x)$

The dataset features are represented as an (I,N) array with with each column $n$ corresponding to a feature vector $\phi^{(n)}$.

<table style="border:1">
    <tr>
        <td>$\phi^{(1)}_1$</td>
        <td>$\phi^{(2)}_1$</td>
        <td>$\dots$</td>
        <td>$\phi^{(N)}_1$</td>
    </tr>
    <tr>
        <td>$\phi^{(1)}_2$</td>
        <td>$\phi^{(2)}_2$</td>
        <td>$\dots$</td>
        <td>$\phi^{(N)}_2$</td>
    </tr>
    <tr>
        <td>$\dots$</td>
        <td>$\dots$</td>
        <td>$\dots$</td>
        <td>$\dots$</td>
    </tr>
    <tr>
        <td>$\phi^{(1)}_I$</td>
        <td>$\phi^{(2)}_I$</td>
        <td>$\dots$</td>
        <td>$\phi^{(N)}_I$</td>
    </tr>
</table>

In [ ]:
def input2feature(input_vec):
    '''
    Args:
        input_vec: Input vector of shape (1,2)
        
    Returns:
        feat_vec: Feature vector of shape(6,1)
    '''
    
    feat_vec = np.zeros((6,1))
    
    feat_vec[0,0] = input_vec[0]
    feat_vec[1,0] = input_vec[0]**2
    feat_vec[2,0] = input_vec[1]
    feat_vec[3,0] = input_vec[1]**2
    feat_vec[4,0] = input_vec[0] * input_vec[1]
    feat_vec[5,0] = 1.0
    
    return feat_vec

In [ ]:
phis = np.zeros((6, N))

for n in range(N):
    
    phis[:,n:n+1] =input2feature(dataset[n,:])

#phis[0,:] = dataset[:,0]                 # x_1
#phis[1,:] = dataset[:,0]**2              # x_1^2
#phis[2,:] = dataset[:,1]                 # x_2
#phis[3,:] = dataset[:,1]**2              # x_2^2
#phis[4,:] = dataset[:,0] * dataset[:,1]  # x_1 * x_2
#phis[5,:] = 1                            # bias

# Number of features
D = phis.shape[0]

ts = np.zeros((1,N))
ts[0,:] = dataset[:,2]

### Labels

Stored as an (1,N) array with each column $n$ corresponding to label $t$ of example $n$.

<table style="border:1">
    <tr>
        <td>$t^{(1)}$</td>
        <td>$t^{(2)}$</td>
        <td>$\dots$</td>
        <td>$t^{(N)}$</td>
    </tr>
</table>

### Summary of data

In [ ]:
print(f"Feature array 'phis' : {phis.shape}")
print(f"Label array 'ts'     : {ts.shape}")
print(f"    #Samples  : {D}")
print(f"    #Features : {D}")

# What is this local variational inference functional again?

In [ ]:
def lambda_func(xi):
    return (expit(xi) - 0.5) / (2*xi + 1e-24)

# Expectations

In [ ]:
def gamma_expectation(a_N, b_N):
    '''Returns the expected value of a Gamma distribution defined by parameters 'a_N' and 'b_N'.
    
    Ref: Bishop Eq 10.182 (p.505)
    '''
    return a_N / b_N


def gaussian_quadratic_expectation_wrt_w(mu_N):
    '''Returns the expected value of the quadratic weight vectors for a Gaussian distribution w.r.t. weights.
    
    Because we are taking the expectations w.r.t. weights, other unrelated terms can be dropped from the general expectation.
    
    Ref: Bishop Eq 10.183 (p.505)
    '''
    return np.matmul(mu_N.T, mu_N)

# Maximization (re-estimation) equations

In [ ]:
#######################################################
#  Distribution 'q(w) = Gaussian(w | mu_N, sigma_N)'
#######################################################

def maximization_mu(phis, ts, sigma_N):
    '''Maximizes the weight vector distribution 'q(w)' w.r.t. mean vector.
    
    Args:
        phis:   : Feature matrix of shape (D,N)
        ts      : Label vector of shape (1,N)
        sigma_N : Covariance matrix for 'q(w)' of shape (D,D)
    
    Returns:
        mu_N : Vector of shape (D,1)
    
    Ref: Bishop Eq 10.175 (p.504)
    '''
    
    # Vector container for summing individual samples
    sample_vec_sum = np.zeros((phis.shape[0], 1))
    
    # Compute and sum each sample individually (TODO: vectorization)
    for i in range(phis.shape[1]):
        
        # Vectors for sample 'i'
        phi_i = phis[:,i:i+1]  # [D,1]
        t_i = ts[:,i].item()   # scalar
        
        sample_vec_sum += (t_i - 0.5) * phi_i
    
    mu_N = np.matmul(sigma_N, sample_vec_sum)
    
    return mu_N


def maximization_sigma_inv(expectation_alpha, xis, phis):
    '''Maximizes the weight vector distribution 'q(w)' w.r.t. variance of data.
    
    Args:
        expectation_alpha : Expected value of Gamma distribution modeling variance of 'q(w)'.
        xis               : Variational variable related to the class certainty of a sample (I think).
        phis              : Feature matrix of shape (D,N).
        
    Returns:
        sigma_inv : Precision of q(w) which optimizes the approximation.
    
    Ref: Bishop Eq 10.176 (p.504)
    '''
    
    # Vector container for summing individual samples
    sample_vec_sum = np.zeros((D,D))
    
    # Compute and sum each sample individually (TODO: vectorization)
    for i in range(phis.shape[1]):
        
        # Vectors for sample 'i'
        phi_i = phis[:,i:i+1]   # [D,1]
        xi_i = xis[:,i].item()  # scalar
        
        sample_vec_sum += lambda_func(xi_i) * np.matmul(phi_i, phi_i.T)
        
    sigma_inv = expectation_alpha * np.eye(D) + 2.0 * sample_vec_sum

    return sigma_inv


####################################################
#  Distribution 'q(alpha) = Gam(alpha | a_N, b_N)  #
####################################################

def maximization_an(a_0, D):
    '''Maximizes the covariance matrix 'q(alpha)' w.r.t. the 'a_N' parameter.
    
    Args:
        a_0 : Value of initial variational hyperpriors.
        D   : Number of features.
    '''
    return a_0 + D


def maximization_bn(b_0, expectation_quadratic_w):
    '''Maximizes the covariance matrix 'q(alpha)' w.r.t. the 'a_N' parameter.
    
    Args:
        b_0                     : Value of initial variational hyperpriors.
        expectation_quadratic_w : The expected value of the quadratic weight vectors for a Gaussian distribution w.r.t. weights.
    '''
    return b_0 + 0.5 * expectation_quadratic_w


###############
#  Xi update
###############

def xi_update(phi, mu_N, sigma_N):
    '''Computes the optimal variational variable related to class certainity (I think) of one sample, according to weight distribution 'q(w)'.
    
    Args:
        phi     : Sample feature vector of shape (D,1)
        mu_N    : Mean vector for weight distribution 'q(w)' of shape (D,1)
        sigma_N : Covariance matrix for weight distribution 'q(w)'  of shape (D,D)
    
    Returns : 'xi' value for one sample given 'q(w)'
    '''
    # xi = phi.T * (sigma_N + mu_N * mu_N.T) * phi
    A = sigma_N + np.matmul(mu_N, mu_N.T)
    xi = np.matmul(np.matmul(phi.T, A), phi)
    xi = np.sqrt(xi)
    return xi


def xis_update(phis, mu_N, sigma_N):
    '''Computes the optimal variational variable related to class certainity (I think) of all samples.
    
    Args:
        phi     : Sample feature matrix of shape (D,N)
        mu_N    : Mean vector for weight distribution 'q(w)' of shape (D,1)
        sigma_N : Covariance matrix for weight distribution 'q(w)' of shape (D,D)
    '''
    N = phis.shape[1]
    
    # Vector container for storing updated 'xi' values
    xis_new = np.zeros((1,N))
    
    # Compute each sample individually
    for i in range(N):
        
        # Vector for sample 'i'
        phi_i = phis[:,i:i+1]   # [D,1]
        
        xis_new[0,i] = xi_update(phi_i, mu_N, sigma_N)
    
    return xis_new



# Expectation Maximization iteration

All variational variables $\xi$ and parameters $\mu_N$. $\Sigma_N$, as well as the variatioanl hyperpriors $a_0$, $b_0$, are first initialized with some random value.

The EM algorithm is implemented to infer the optimal distributional parameters for $q(w)$ and $q(\alpha)$ which maximizes the approximation of the joint distribution $p(t, x, w)$ where $(t, x)$ denotes the data.

_Expectation step:_

Given the current "best estimation" of the distributions $q(w)$ and $q(\alpha)$, compute expectations for variables according to the distributions.

_Maximization step:_

Given the expectations computed in the previous step, re-optimize the parameters for the same distributions, as well as the variational variable $\xi$.

In [ ]:
#######################
#  Initial variables
#######################

# Hyperpriors for Gamma distribution
a_0 = 1.0
b_0 = 1.0

a_N = a_0
b_N = b_0

mu_N = np.random.random((D,1))
sigma_N = np.random.random((D,D))

# Sample-specific latent variables 'xi'
xis = np.ones((1, N))

old_vals = np.zeros(7)

deltas = np.zeros((7,100))

for iter_idx in range(100):
    
    # Expectation step
    expectation_alpha = gamma_expectation(a_N, b_N)                             # scalar
    expectation_quadratic_w_wrt_w = gaussian_quadratic_expectation_wrt_w(mu_N)  # scalar ?
    
    # Maximization step
    a_N = maximization_an(a_0, D)                                               # scalar
    b_N = maximization_bn(b_0, expectation_quadratic_w_wrt_w)                   # scalar
    sigma_N_inv = maximization_sigma_inv(expectation_alpha, xis, phis)          # [D,D]
    sigma_N = np.linalg.inv(sigma_N_inv)                                        # [D,D]
    mu_N = maximization_mu(phis, ts, sigma_N)                                   # [D,1]
    xis = xis_update(phis, mu_N, sigma_N)                                       # [1,N]
    

### Computing the predictive posterior $p(t^{new} | \phi^{new}, \theta)$



In [ ]:
def predictive_posterior_prob(x, mu_n, sigma_N):
    '''Computes the probability of a new input vector 'x' belonging to class 1 using the learned variational parameters.
    
    Args:
        x       : Input vector (or list) of shape (1,2)
        mu_n    : Learned mean vector for weight distribution 'q(w)'
        sigma_N : Learned covariance matrix for weight distribution 'q(w)'
        
    Ref: Bishop Eq 4.155 (p.220)
    '''
    phi_new =input2feature(x)

    mu_a = np.matmul(mu_N.T, phi_new).item()
    sigma_a_2 = np.matmul(np.matmul(phi_new.T, sigma_N), phi_new).item()

    kappa = np.power(1.0 + np.pi/8.0 * sigma_a_2,-0.5)

    p = expit(kappa*mu_a)
    
    return p

In [ ]:
# Compute posterior class probability over the whole input space.

x = np.arange(-10, 10, 0.1)
y = np.arange(-10, 10, 0.1)

xx, yy = np.meshgrid(x, y, sparse=False)

p_array = np.zeros(xx.shape)

for i in range(p_array.shape[0]):
    for j in range(p_array.shape[1]):
        
        x = [xx[i,j], yy[i,j]]
        
        p_array[i,j] = predictive_posterior_prob(x, mu_N, sigma_N)
        

plt.figure(figsize=(10,10))
plt.imshow(p_array, origin='lower', extent=[-10,10,-10,10])
plt.scatter(gaussian_1[:,0], gaussian_1[:,1])
plt.scatter(gaussian_2[:,0], gaussian_2[:,1])
plt.xlim(-10,10)
plt.ylim(-10,10)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(p_array, origin='lower', extent=[-10,10,-10,10])
plt.scatter(gaussian[:,0], gaussian[:,1])
plt.scatter(xs, ys)
plt.xlim(-10,10)
plt.ylim(-10,10)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()